In [23]:
import os

In [24]:
%pwd

'c:\\Users\\hp\\Documents\\machine_leanrning\\Deep Learning\\MLOPS\\Chicken-Disease-Classifier-With-Mlflow-and-DVC\\research'

In [25]:
os.chdir("../")

In [26]:
%pwd

'c:\\Users\\hp\\Documents\\machine_leanrning\\Deep Learning\\MLOPS\\Chicken-Disease-Classifier-With-Mlflow-and-DVC'

In [31]:
from dataclasses import dataclass
from pathlib import Path



In [ ]:
# update entity

@dataclass(frozen=True)
class PrepareBaseModelConfig:
    root_dir: Path
    base_model_path: Path
    updated_base_model_path: Path
    params_image_size: list          # Should match your params.yaml
    params_learning_rate: float
    params_include_top: bool
    params_weights: str
    params_classes: int


### 02 base model

In [33]:
# prepare the configuration manager

from ChickenDiseaseClassifier.constants import *
from ChickenDiseaseClassifier.utils.common import read_yaml, create_directories


In [ ]:
# from pathlib import Path

# CONFIG_FILE_PATH = Path("../config/config.yaml")
# PARAM_FILE_PATH = Path("../params.yaml")


In [45]:
from src.ChickenDiseaseClassifier.constants import *
# from src.ChickenDiseaseClassifier.constants import CONFIG_FILE_PATH, PARAMS_FILE_PATH
from src.ChickenDiseaseClassifier.utils.common import read_yaml, create_directories

In [43]:
# %ddir

In [51]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath=CONFIG_FILE_PATH,   # from constants
        params_filepath=PARAMS_FILE_PATH    # notice the S
    ):
        # Load YAMLs
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        # Make artifacts directory if missing
        create_directories([self.config.artifacts_root])
    
    
    def get_prepare_base_model_config(self) -> PrepareBaseModelConfig:
        config = self.config.prepare_base_model
        
        create_directories([config.root_dir])

        prepare_base_model_config = PrepareBaseModelConfig(
            root_dir=Path(config.root_dir),
            base_model_path=Path(config.base_model_path),
            updated_base_model_path=Path(config.updated_base_model_path),
            params_image_size=self.params.IMAGE_SIZE,
            params_learning_rate=self.params.LEARNING_RATE,
            params_include_top=self.params.INCLUDE_TOP,
            params_weights=self.params.WEIGHTS,
            params_classes=self.params.CLASSES
        )

        return prepare_base_model_config

In [21]:
%pwd

'c:\\Users\\hp\\Documents\\machine_leanrning\\Deep Learning\\MLOPS\\Chicken-Disease-Classifier-With-Mlflow-and-DVC\\research'

In [47]:
# update component

import os
import urllib.request as request
from zipfile import ZipFile
import tensorflow as tf

In [58]:
class PrepareBaseModel:
    def __init__(self, config: PrepareBaseModelConfig):
        self.config = config

    
    def get_base_model(self):
        self.model = tf.keras.applications.MobileNetV2(
            input_shape=self.config.params_image_size,
            weights=self.config.params_weights,
            include_top=self.config.params_include_top
        )

        self.save_model(path=self.config.base_model_path, model=self.model)


    
    @staticmethod
    def _prepare_full_model(model, classes, freeze_all, freeze_till, learning_rate):
        if freeze_all:
            for layer in model.layers:
                model.trainable = False
        elif (freeze_till is not None) and (freeze_till > 0):
            for layer in model.layers[:-freeze_till]:
                model.trainable = False

        flatten_in = tf.keras.layers.Flatten()(model.output)
        prediction = tf.keras.layers.Dense(
            units=classes,
            activation="softmax"
        )(flatten_in)

        full_model = tf.keras.models.Model(
            inputs=model.input,
            outputs=prediction
        )

        full_model.compile(
            optimizer=tf.keras.optimizers.SGD(learning_rate=learning_rate),
            loss=tf.keras.losses.CategoricalCrossentropy(),
            metrics=["accuracy"]
        )

        full_model.summary()
        return full_model
    

    def update_base_model(self):
        self.full_model = self._prepare_full_model(
            model=self.model,
            classes=self.config.params_classes,
            freeze_all=True,
            freeze_till=None,
            learning_rate=self.config.params_learning_rate
        )

        self.save_model(path=self.config.updated_base_model_path, model=self.full_model)
    


    @staticmethod
    def save_model(path: Path, model: tf.keras.Model):
        model.save(path)

In [59]:
try:
    config = ConfigurationManager()
    prepare_base_model_config = config.get_prepare_base_model_config()
    prepare_base_model = PrepareBaseModel(config=prepare_base_model_config)
    prepare_base_model.get_base_model()
    prepare_base_model.update_base_model()
except Exception as e:
    raise e

[2026-01-01 11:14:51,943]: INFO: common: yaml file: config\config.yaml loaded successfully:
[2026-01-01 11:14:51,997]: INFO: common: yaml file: params.yaml loaded successfully:
[2026-01-01 11:14:52,003]: INFO: common: created directory at: artifacts:
[2026-01-01 11:14:52,009]: INFO: common: created directory at: artifacts/prepare_base_model:
9406464/9406464 [==============================] - 5s 1us/step
[2026-01-01 11:15:03,969]: WARNING: saving_utils: Compiled the loaded model, but the compiled metrics have yet to be built. `model.compile_metrics` will be empty until you train or evaluate the model.:
Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_2 (InputLayer)           [(None, 128, 128, 3  0           []                               
                                )]                                             